In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

# ========================================
# 1. 병원 데이터 로드
# ========================================
# CSV 파일 경로를 본인 경로로 수정하세요
hospitals = pd.read_csv('D:/emergency_room/data-analysis/data/raw/busan_hospitals_static.csv')

print(f"병원 수: {len(hospitals)}")
print(f"병원 목록:\n{hospitals[['id', 'name', 'emergency_level', 'beds_total']].head()}")

# ========================================
# 2. 시간대별 기본 혼잡도 패턴 정의
# ========================================
def get_base_congestion(hour):
    """
    시간대별 기본 대기시간 (분)
    실제 응급실 이용 패턴 반영
    """
    patterns = {
        # 새벽 (0-5시): 여유
        0: 15, 1: 12, 2: 10, 3: 10, 4: 12, 5: 15,
        # 오전 (6-11시): 점차 증가
        6: 25, 7: 30, 8: 35, 9: 40, 10: 45, 11: 50,
        # 점심 (12-13시): 매우 혼잡
        12: 55, 13: 50,
        # 오후 (14-17시): 보통
        14: 40, 15: 35, 16: 40, 17: 45,
        # 저녁 (18-21시): 매우 혼잡 (퇴근 시간)
        18: 60, 19: 65, 20: 60, 21: 50,
        # 밤 (22-23시): 감소
        22: 35, 23: 25
    }
    return patterns[hour]

# ========================================
# 3. 병원 특성에 따른 혼잡도 조정
# ========================================
def adjust_by_hospital(base_congestion, emergency_level, beds_total):
    """
    병원 특성을 반영한 혼잡도 조정
    - 등급이 높을수록 환자 많음
    - 병상이 많을수록 처리 능력 높음
    """
    # 응급실 등급별 가중치
    level_multiplier = {
        '권역응급의료센터': 1.3,      # 30% 더 혼잡
        '지역응급의료센터': 1.1,      # 10% 더 혼잡
        '지역응급의료기관': 0.9,      # 10% 덜 혼잡
        '응급의료기관': 0.8           # 20% 덜 혼잡
    }
    
    multiplier = level_multiplier.get(emergency_level, 1.0)
    adjusted = base_congestion * multiplier
    
    # 병상 수에 따른 조정 (병상이 많으면 대기시간 감소)
    if pd.notna(beds_total):
        if beds_total > 500:
            adjusted *= 0.85  # 대형병원: 15% 감소
        elif beds_total > 300:
            adjusted *= 0.95  # 중형병원: 5% 감소
        elif beds_total < 200:
            adjusted *= 1.1   # 소형병원: 10% 증가
    
    return int(adjusted)

# ========================================
# 4. 혼잡도 레벨 및 색상 결정
# ========================================
def get_congestion_info(wait_time):
    """
    대기시간 → 혼잡도 레벨 & 색상
    """
    if wait_time < 30:
        return '여유', 'green'
    elif wait_time < 60:
        return '보통', 'yellow'
    else:
        return '혼잡', 'red'

# ========================================
# 5. 예측 데이터 생성
# ========================================
predictions = []

for _, hospital in hospitals.iterrows():
    hospital_id = hospital['id']
    emergency_level = hospital['emergency_level']
    beds_total = hospital['beds_total']
    
    for hour in range(24):
        # 기본 혼잡도
        base = get_base_congestion(hour)
        
        # 병원 특성 반영
        adjusted = adjust_by_hospital(base, emergency_level, beds_total)
        
        # 랜덤 변동 추가 (±10분)
        random_noise = np.random.randint(-10, 11)
        final_wait_time = max(5, adjusted + random_noise)  # 최소 5분
        
        # 혼잡도 레벨 & 색상
        level, color = get_congestion_info(final_wait_time)
        
        predictions.append({
            'hospital_id': hospital_id,
            'prediction_hour': hour,
            'predicted_wait_time': final_wait_time,
            'congestion_level': level,
            'congestion_color': color
        })

# DataFrame 생성
df_predictions = pd.DataFrame(predictions)

print(f"\n생성된 예측 데이터 수: {len(df_predictions)}")
print(f"\n데이터 샘플:\n{df_predictions.head(10)}")

# ========================================
# 6. 통계 확인
# ========================================
print("\n=== 혼잡도 분포 ===")
print(df_predictions['congestion_level'].value_counts())

print("\n=== 시간대별 평균 대기시간 ===")
hourly_avg = df_predictions.groupby('prediction_hour')['predicted_wait_time'].mean()
print(hourly_avg.round(1))

# ========================================
# 7. CSV 파일 저장
# ========================================
output_path = 'D:/emergency_room/data-analysis/data/simulated/emergency_predictions.csv'
df_predictions.to_csv(output_path, index=False, encoding='utf-8-sig')

print(f"\n✅ CSV 파일 저장 완료: {output_path}")
print(f"총 {len(df_predictions)}개의 예측 데이터 생성")

병원 수: 46
병원 목록:
      id        name emergency_level  beds_total
0  ER001   고신대학교복음병원        지역응급의료기관         844
1  ER002  인제대학교부산백병원        지역응급의료기관         810
2  ER003     부산대학교병원        지역응급의료기관        1040
3  ER004     동아대학교병원        지역응급의료기관         973
4  ER005   학교법인 춘해병원        지역응급의료기관         201

생성된 예측 데이터 수: 1104

데이터 샘플:
  hospital_id  prediction_hour  predicted_wait_time congestion_level  \
0       ER001                0                    9               여유   
1       ER001                1                    9               여유   
2       ER001                2                    6               여유   
3       ER001                3                    6               여유   
4       ER001                4                    5               여유   
5       ER001                5                   12               여유   
6       ER001                6                   21               여유   
7       ER001                7                   24               여유   
8       ER001